# Run Moatless Tools

First, index the codebase in a vector store.

Set `repo_dir` to the path of the repository you want to index.

In [2]:
from moatless.index import CodeIndex, IndexSettings
from moatless.repository import FileRepository
from dotenv import load_dotenv
import os


load_dotenv()
# An OPENAI_API_KEY is required to use the OpenAI Models
index_settings = IndexSettings(
    embed_model="text-embedding-3-small",
)

repo_dir = "/tmp/moatless-tools"
persist_dir = "/tmp/moatless-tools/.index"

file_repo = FileRepository(repo_path=repo_dir)

if os.path.exists(persist_dir):
    print(f"Found existing index on {persist_dir}")
    code_index = CodeIndex.from_persist_dir(persist_dir, file_repo=file_repo)
else:
    code_index = CodeIndex(file_repo=file_repo, settings=index_settings)
    nodes, tokens = code_index.run_ingestion()
    print(f"Indexed {nodes} nodes and {tokens} tokens")
    code_index.persist(persist_dir)


Found existing index on /tmp/moatless-tools/.index


Then use the `SearchLoop` to find the relevant code.

In [21]:
from moatless.loop import AgenticLoop
from moatless.completion.base import BaseCompletionModel, LLMResponseFormat
from moatless.schema import MessageHistoryType
from moatless.agent.code_agent import CodingAgent
import json

instructions = "Make number of retries in the completion handling configurable"


model = "gpt-4o"

completion_model = BaseCompletionModel.create(
    model=model,
    temperature=0.0,
    response_format=LLMResponseFormat.TOOLS,
    thoughts_in_action=True
)

agent = CodingAgent.create(
    completion_model=completion_model,
    repository=file_repo,
    code_index=code_index,
    message_history_type=MessageHistoryType.MESSAGES,
    thoughts_in_action=True,
)

def event_handler(event):
    if event["event_type"] == "loop_iteration":
        data = event["data"]
        print(f"Iteration {data['iteration']}")
        if data.get("action_steps"):
            for action_step in data.get("action_steps"):
                print("\n\n========================== Action ==========================")
                print(json.dumps(action_step['action'], indent=2))
                print("\n========================== Observation ==========================")
                print(action_step["observation"]["message"])
    else:
        print(f"{event['event_type']}")

loop = AgenticLoop.create(
    instructions, agent=agent, repository=file_repo, max_iterations=20
)
loop.add_event_handler(event_handler)

loop.run()


loop_started
Iteration 2


========================== Action ==========================
{
  "thoughts": "I need to locate the code responsible for handling completion retries to make it configurable.",
  "file_pattern": null,
  "query": "completion handling retries",
  "category": "implementation",
  "action_args_class": "moatless.actions.semantic_search.SemanticSearchArgs"
}

========================== Observation ==========================
Found the following relevant code:
moatless/completion/base.py
```
     1	import json
     2	import logging
     3	import os
     4	from abc import ABC, abstractmethod
     5	from enum import Enum
     6	from typing import Optional, List, Union, Any, Dict
     7	
     8	import tenacity
     9	from pydantic import BaseModel, Field, model_validator
    10	
    11	from moatless.completion.model import Completion, Usage
    12	from moatless.completion.schema import ResponseSchema, AllMessageValues, ChatCompletionCachedContent
    13	from moatless.excep

Generate the git patch from the last node to see the result

In [22]:
print(loop.get_last_node().file_context.generate_git_patch())

--- a/moatless/completion/base.py
+++ b/moatless/completion/base.py
@@ -102,6 +102,7 @@
         default=None, description="The system prompt to use for completion", exclude=True
     )
 
+    retry_attempts: int = Field(3, description="The number of retry attempts for completion")
     _completion_params: Optional[Dict[str, Union[str, Dict, List]]] = None
     _initialized: bool = False
 
@@ -213,7 +214,7 @@
 
         @tenacity.retry(
             retry=tenacity.retry_if_exception_type((CompletionRetryError)),
-            stop=tenacity.stop_after_attempt(3),
+            stop=tenacity.stop_after_attempt(self.retry_attempts),
             wait=tenacity.wait_fixed(0),
             reraise=True,
             before_sleep=lambda retry_state: logger.warning(

--- a/tests/completion/test_tool_call.py
+++ b/tests/completion/test_tool_call.py
@@ -481,3 +481,45 @@
     assert exc_info.value.accumulated_usage is not None
     assert exc_info.value.accumulated_usage.prompt_tokens == 60  # 10 +

Execute the `CodeLoop` to apply the changes.

Run a `$ git diff` to see the changes.